In [32]:
import numpy as np
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
%precision 3

'%.3f'

In [2]:
#Load Data from pickle objects

# CDC demographic - Body weight
cdc_data = pd.read_pickle('cdc_demographic_2008.pkl')

# Pregnancy Data
data = pd.read_pickle('nsfg_data.pkl')
femResp = pd.read_pickle('fem_resp.pkl')

In [3]:
males = cdc_data[cdc_data.sex == 1 ]
females = cdc_data[cdc_data.sex == 2 ]

# successful pregnacies
live = data[data.outcome == 1]
# full term pregnacies
full_term = live[live.wksgest > 36]
# firstborns
firsts = live[live.birthord == 1] 
# subsequent births
others = live[live.birthord != 1]

### 11.1 Simple Regression

In [4]:
formula = 'totalwgt_lb ~ agepreg'
model = smf.ols(formula, data=live)
results = model.fit()

In [5]:
# intercept 
results.params['Intercept']

6.83

In [6]:
# slope 
results.params['agepreg']

0.02

In [7]:
#slope_pvalue
results.pvalues['agepreg']

0.00

In [8]:
# coefficient of Determination
results.rsquared

0.00

In [9]:
# model p-value
results.f_pvalue

0.00

In [10]:
# model fit summary
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     43.02
Date:                Mon, 08 Jan 2018   Prob (F-statistic):           5.72e-11
Time:                        16:12:30   Log-Likelihood:                -15897.
No. Observations:                9038   AIC:                         3.180e+04
Df Residuals:                    9036   BIC:                         3.181e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.8304      0.068    100.470      0.000       6.697       6.964
agepreg        0.0175      0.003      6.559      0.000       0.012       0.023
==============================================================================
Omnibus:                     1024.052   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3081.833
Skew:                          -0.601   Prob(JB):                         0.00
Kurtosis:                       5.596   Cond. No.                         118.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
#%% Check for correlation between mother's age & birth weight

diff_weight = firsts.totalwgt_lb.mean() - others.totalwgt_lb.mean()
diff_age = firsts.agepreg.mean() - others.agepreg.mean()

results = smf.ols('totalwgt_lb ~ agepreg', data=live).fit()
slope = results.params['agepreg']
slope * diff_age # about half of the observed difference

-0.06

### 11.2 Multiple regression

In [12]:
live['isfirst'] = live.birthord == 1
formula = 'totalwgt_lb ~ isfirst + agepreg'
results = smf.ols(formula, data=live).fit()

results.summary()

/archive/Studies/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     24.02
Date:                Mon, 08 Jan 2018   Prob (F-statistic):           3.95e-11
Time:                        16:13:32   Log-Likelihood:                -15894.
No. Observations:                9038   AIC:                         3.179e+04
Df Residuals:                    9035   BIC:                         3.182e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           6.9142      0.078     89.073      0.000       6.762       7.066
isfirst[T.True]    -0.0698      0.031     -2.236      0.025      -0.131      -0.009
agepreg             0.0154      0.003      5.499      0.000       0.010       0.021
==============================================================================
Omnibus:                     1019.945   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3063.682
Skew:                          -0.599   Prob(JB):                         0.00
Kurtosis:                       5.588   Cond. No.                         137.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### 11.3 Nonlinear relationships

In [13]:
live['agepreg2'] = live.agepreg**2
formula = 'totalwgt_lb ~ isfirst + agepreg + agepreg2'

results = smf.ols(formula, data=live).fit()

results.summary()

/archive/Studies/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     22.64
Date:                Mon, 08 Jan 2018   Prob (F-statistic):           1.35e-14
Time:                        16:14:44   Log-Likelihood:                -15884.
No. Observations:                9038   AIC:                         3.178e+04
Df Residuals:                    9034   BIC:                         3.181e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           5.6923      0.286     19.937      0.000       5.133       6.252
isfirst[T.True]    -0.0504      0.031     -1.602      0.109      -0.112       0.011
agepreg             0.1124      0.022      5.113      0.000       0.069       0.155
agepreg2           -0.0018      0.000     -4.447      0.000      -0.003      -0.001
==============================================================================
Omnibus:                     1007.149   Durbin-Watson:                   1.616
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3003.343
Skew:                          -0.594   Prob(JB):                         0.00
Kurtosis:                       5.562   Cond. No.                     1.39e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.39e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### 11.4 Data mining

In [14]:
# check for outliers
live.prglngth.value_counts().sort_index()
# exclude outliers
live = live[(live.prglngth>30) & (live.prglngth<46)]
femResp.index = femResp.caseid
# pandas left join oeeration
join = live.join(femResp, on='caseid', rsuffix='_fem')


'''For each variable we construct a model, compute Rsquared, and append the results
to a list. The models all include agepreg, since we already know that it has
some predictive power.'''


In [16]:
t = []
for name in join.columns:
    try:
        if join[name].var(skipna=True) < 1e-7:
            continue
        
        formula = ('totalwgt_lb ~ ' +  name)
        model = smf.ols(formula, data=join)
        if model.nobs < len(join)/2:
            continue

        results = model.fit()
    except (ValueError, TypeError):
        continue

    t.append((results.rsquared, name))


In [20]:
# sort the results and select the variables that yield the highest values of Rsquared.

t.sort(reverse=True)
print(f'Name  \t\t  MeanSquaredError')
for mse, name in t[:20]:
    print(f'{name}  \t\t  {mse}')

Name  		  MeanSquaredError
totalwgt_lb  		  1.0
birthwgt_lb  		  0.9497063506323615
lbw1  		  0.2953302391941195
prglngth  		  0.1318057082742845
wksgest  		  0.12329150047839943
mosgest  		  0.022386402593244092
babysex  		  0.01318171082742614
race_fem  		  0.01253897164309914
race  		  0.01253897164309914
paydu  		  0.012011033980170671
rmarout03  		  0.011361648429711213
rmarout6  		  0.010493503028199402
totincr  		  0.010131159799792178
nbrnaliv  		  0.01010053613134465
fmarcon5  		  0.010068852563572595
anynurse  		  0.010024511481842469
marout03  		  0.009563360926018172
fmarout5  		  0.009496568136071248
marcon03  		  0.009362348277173416
rmarout01  		  0.009312466365693162


In [21]:
#%% Final Formula
'''
- C(race) tells the formula parser (Patsy) to treat race as a categorical variable,
- writing babysex==1 converts it to boolean, True for male and false for female.
- nbrnaliv==1 is True for single births and paydu==1 is True for respondents who own their houses.
- rmarout specifies informal marital status ~ cohabitation 
'''

formula = ('totalwgt_lb ~ babysex==1 +  C(race) + nbrnaliv==1 + fmarital==1' )
results = smf.ols(formula, data=join).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.054
Method:                 Least Squares   F-statistic:                     100.6
Date:                Mon, 08 Jan 2018   Prob (F-statistic):          1.70e-103
Time:                        16:20:58   Log-Likelihood:                -14293.
No. Observations:                8770   AIC:                         2.860e+04
Df Residuals:                    8764   BIC:                         2.864e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 5.5195      0.110     50.043      0.000       5.303       5.736
babysex == 1[T.True]      0.2952      0.026     11.189      0.000       0.243       0.347
C(race)[T.2]              0.4001      0.032     12.521      0.000       0.337       0.463
C(race)[T.3]              0.2635      0.052      5.072      0.000       0.162       0.365
nbrnaliv == 1[T.True]     1.3720      0.108     12.710      0.000       1.160       1.584
fmarital == 1[T.True]     0.1023      0.028      3.673      0.000       0.048       0.157
==============================================================================
Omnibus:                      380.129   Durbin-Watson:                   1.593
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1278.767
Skew:                          -0.033   Prob(JB):                    2.09e-278
Kurtosis:                       4.870   Cond. No.                         20.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:
# RMSE without using model
live.totalwgt_lb.std()
# RMSE with model 
results.resid.std() # not satisfactory

1.23

### 11.6 Logistic regression

In [23]:
# exclude outliers
df = live[(live.prglngth>30) & (live.prglngth<46)]

# convert the dependent variable to binary type
df['boy'] = (df.babysex==1).astype(int)

model = smf.logit('boy ~ agepreg', data=df)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.693019
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 8873
Model:                          Logit   Df Residuals:                     8871
Method:                           MLE   Df Model:                            1
Date:                Mon, 08 Jan 2018   Pseudo R-squ.:               7.217e-06
Time:                        16:21:35   Log-Likelihood:                -6149.2
converged:                       True   LL-Null:                       -6149.2
                                        LLR p-value:                    0.7658
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0030      0.098      0.030      0.976      -0.188       0.194
agepreg        0.0011      0.004      0.298      0.766      -0.006       0.009
==============================================================================
"""

'''
The result is a Logit object that represents the model. 
It contains attributes called :
- endog = endogenous/dependent variable, 
- exog =  exogenous/explanatory variables.

'''

In [24]:
endog = pd.DataFrame(model.endog, columns=[model.endog_names])
exog = pd.DataFrame(model.exog, columns=model.exog_names)

### Model that includes several factors believed to be associated with sex ratio

In [25]:
formula = 'boy ~ agepreg + hpagelb + birthord + C(race)'
model = smf.logit(formula, data=df)
results = model.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.692953
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 8774
Model:                          Logit   Df Residuals:                     8768
Method:                           MLE   Df Model:                            5
Date:                Mon, 08 Jan 2018   Pseudo R-squ.:               0.0001403
Time:                        16:23:03   Log-Likelihood:                -6080.0
converged:                       True   LL-Null:                       -6080.8
                                        LLR p-value:                    0.8881
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.0317      0.104     -0.305      0.760      -0.235       0.172
C(race)[T.2]    -0.0208      0.051     -0.409      0.683      -0.121       0.079
C(race)[T.3]     0.0016      0.083      0.019      0.985      -0.161       0.165
agepreg         -0.0026      0.006     -0.477      0.633      -0.013       0.008
hpagelb          0.0046      0.004      1.078      0.281      -0.004       0.013
birthord         0.0064      0.022      0.288      0.773      -0.037       0.050
================================================================================
"""

In [26]:
endog = pd.DataFrame(model.endog, columns=[model.endog_names])
exog = pd.DataFrame(model.exog, columns=model.exog_names)

#### Accuracy of the model: the number of successful predictions, compared with what we would expect by chance.

In [31]:
actual = endog['boy']
baseline = actual.mean()
baseline

0.51

In [27]:
# results.predict returns a NumPy array of probabilities, which we round off to 0 or 1.
predict = (results.predict() >= 0.5)
true_pos = predict * actual
true_neg = (1 - predict) * (1 - actual)

In [28]:
# Accuracy is the fraction of correct guesses:
acc = (sum(true_pos) + sum(true_neg)) / len(actual)


### Using the model to make a prediction for the office pool.

In [29]:
'''
Suppose your friend is 35 years old and white, her husband is 39, and they
are expecting their third child:
'''

columns = ['agepreg', 'hpagelb', 'birthord', 'race']
new = pd.DataFrame([[35, 39, 3, 2]], columns=columns)
y = results.predict(new)